In [1]:
import pandas as pd
from datetime import date

## PARÁMETROS

In [2]:
N_RETOS = 5
TOMAR_RETOS = True #True si se desea que se tengan en cuenta
INCLUIR_CANCELADOS = True

COLUMNA_CURSOS = 'GRUPO_PROGRAMACION '
NOMBRE_INFORME = 'INFORME_SEMANAL_19_JUL.xlsx'
NOMBRE_PAGINA_BD = 'BD'
FILAS_ENCABEZADO = 1

COLUMNA_CANCELADO = 'TRIPULANTE_CANCELA'

## LECTURA DE DATOS

In [3]:
# Lectura de datos y quitado de encabezado
DATOS = pd.read_excel('./INPUT/' + NOMBRE_INFORME, engine = 'openpyxl', skiprows=FILAS_ENCABEZADO)
DATOS_COPIA = DATOS.sort_values(by=[COLUMNA_CURSOS], ignore_index=True)
DATOS_COPIA_BD = pd.read_excel('./INPUT/' + NOMBRE_INFORME, engine = 'openpyxl', sheet_name=NOMBRE_PAGINA_BD)

C:\Users\radou\AppData\Roaming\Python\Python39\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
DATOS_COPIA[COLUMNA_CURSOS].unique()

array(['H1', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17',
       'H18', 'H19', 'H2', 'H20', 'H21', 'H22', 'H23', 'H24', 'H25',
       'H26', 'H27', 'H28', 'H29', 'H3', 'H30', 'H31', 'H32', 'H33',
       'H34', 'H35', 'H36', 'H37', 'H38', 'H39', 'H4', 'H40', 'H41',
       'H42', 'H43', 'H44', 'H45', 'H46', 'H47', 'H48', 'H49', 'H5',
       'H50', 'H51', 'H52', 'H53', 'H54', 'H55', 'H56', 'H57', 'H58',
       'H59', 'H6', 'H60', 'H61', 'H62', 'H63', 'H64', 'H65', 'H66',
       'H67', 'H68', 'H69', 'H7', 'H8', 'H9'], dtype=object)

## PROCESAMIENTO INFORME GENERAL

In [5]:
# Estructura de datos
resultado = {
    'CURSO': [],
    'NOMBRE_DOCENTE': [],
    'CORREO_DOCENTE': [],
    'TOTAL_ESTUDIANTES': [],
    'ASISTENCIA_PROMEDIO': [],
}
if INCLUIR_CANCELADOS:
    resultado['ESTUDIANTES_CANCELADOS'] = []
if TOMAR_RETOS:
    for n in range(0, N_RETOS + 1):
        resultado['N_EST_RETO_' + str(n)] = []
        resultado['N_EST_RETO_%_' + str(n)] = []
# Procesamiento
cursos = DATOS_COPIA[COLUMNA_CURSOS].unique()
for curso in cursos:
    resultado['CURSO'].append(curso)
    df = DATOS_COPIA[DATOS_COPIA[COLUMNA_CURSOS] == curso]
    df.to_excel('./OUTPUT/LISTAS/' + curso + '.xlsx', index = False)
    dfBd = DATOS_COPIA_BD[DATOS_COPIA_BD['GRUPO'] == curso]
    totalEst = len(df)
    resultado['NOMBRE_DOCENTE'].append(df['Docente_de_Java'].iloc[0])
    resultado['CORREO_DOCENTE'].append(dfBd['CORREO_DOCENTE_JAVA'].iloc[0])
    resultado['TOTAL_ESTUDIANTES'].append(totalEst)
    resultado['ASISTENCIA_PROMEDIO'].append(df['PORCENTAJE_ASISTENCIA'].mean())
    
    if INCLUIR_CANCELADOS:
        resultado['ESTUDIANTES_CANCELADOS'].append(len(df[df[COLUMNA_CANCELADO] == 'SÍ']))
    
    if TOMAR_RETOS:
        conteosRetos = df['RETOS_FINALIZADOS'].value_counts()
        for n in range(0, N_RETOS + 1):
            cont = conteosRetos.get(n, 0)
            resultado['N_EST_RETO_' + str(n)].append(cont)
            resultado['N_EST_RETO_%_' + str(n)].append(cont/totalEst)

## GUARDADO

In [6]:
pd.DataFrame(resultado).to_excel('./OUTPUT/INFORME_ASISTENCIAS_Y_RETOS_' + str(date.today()) + '.xlsx', index = False)